In [1]:
!pip install redis==2.10.5

In [2]:
!pip install redis

In [1]:
import redis

In [2]:
r = redis.Redis(host='127.0.0.1', port = 6379, db = 0)

In [3]:
r.set("Bahamas", "Nassau")
r.get("Bahamas")

b'Nassau'

In [4]:
r.mset({"Croatia": "Zagreb", "Bahamas": "Nassau"})
r.get('Bahamas')

b'Nassau'

In [5]:
r.hset("realpython", "url", "https://realpython.com/")

0

In [6]:
import random

random.seed(444)
hats = {f"hat:{random.getrandbits(32)}": i for i in (
    {
        "color": "black",
        "price": 49.99,
        "style": "fitted",
        "quantity": 1000,
        "npurchased": 0,
    },
    {
        "color": "maroon",
        "price": 59.99,
        "style": "hipster",
        "quantity": 500,
        "npurchased": 0,
    },
    {
        "color": "green",
        "price": 99.99,
        "style": "baseball",
        "quantity": 200,
        "npurchased": 0,
    })
}

In [7]:
r = redis.Redis(db=1)

In [8]:
import json


In [11]:
with r.pipeline() as pipe:
    for h_id, hat in hats.items():
        pipe.hset(h_id, "color",hat["color"])
        pipe.hset(h_id, "price",hat["price"])
        pipe.hset(h_id, "style",hat["style"])
        pipe.hset(h_id, "quantity",hat["quantity"])
        pipe.hset(h_id, "npurchased", hat["npurchased"])
    pipe.execute()

In [9]:
with r.pipeline() as pipe:
    for h_id, hat in hats.items():
        pipe.hmset(h_id, hat)
    pipe.execute()

In [13]:
r.bgsave()

True

In [11]:
print(r.hgetall("hat:56854717"))

{b'color': b'green', b'price': b'99.99', b'style': b'baseball', b'quantity': b'200', b'npurchased': b'0'}


In [15]:
r.hincrby("hat:56854717", "quantity", -1)

199

In [16]:
r.hget("hat:56854717", "quantity")

b'199'

In [17]:
r.hincrby("hat:56854717", "npurchased", 1)

1

In [18]:
r.hget("hat:56854717", "quantity")

b'199'

In [19]:
import logging
logging.basicConfig()


class OutOfStockError(Exception):
    """Raised when PyHats.com is all out of today's hottest hat"""


def buyitem(r: redis.Redis, itemid: int) -> None:
    with r.pipeline() as pipe:
        error_count = 0
        while True:
            try:
                # Get available inventory, watching for changes
                # related to this itemid before the transaction
                pipe.watch(itemid)
                nleft: bytes = r.hget(itemid, "quantity")
                if nleft > b"0":
                    pipe.multi()
                    pipe.hincrby(itemid, "quantity", -1)
                    pipe.hincrby(itemid, "npurchased", 1)
                    pipe.execute()
                    break
                else:
                    # Stop watching the itemid and raise to break out
                    pipe.unwatch()
                    raise OutOfStockError(
                        f"Sorry, {itemid} is out of stock!"
                    )
            except redis.WatchError:
                # Log total num. of errors by this user to buy this item,
                # then try the same process again of WATCH/HGET/MULTI/EXEC
                error_count += 1
                logging.warning(
                    "WatchError #%d: %s; retrying",
                    error_count, itemid
                )
    return None

In [20]:
buyitem(r, "hat:56854717")
buyitem(r, "hat:56854717")
buyitem(r, "hat:56854717")
r.hmget("hat:56854717", "quantity", "npurchased") 

[b'196', b'4']

In [21]:
for _ in range(196):
    buyitem(r, "hat:56854717")
r.hmget("hat:56854717", "quantity", "npurchased")

[b'0', b'200']

In [22]:
buyitem(r, "hat:56854717")


OutOfStockError: Sorry, hat:56854717 is out of stock!